## calculate the Metabolic Index for each grid point at each depth for FOSI hindcast

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
filename = f'{path}/DPLE_driftcorrected_TEMP_mean_ens_mean.nc'
ds1 = xr.open_dataset(filename)
Temp = ds1['TEMP'].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
#
filename = f'{path}/DPLE_driftcorrected_pO2_mean_dimension_reduced_ens_mean_monthly.nc'
ds2 = xr.open_dataset(filename)
pO2 = ds2['pO2']
Temp

<xarray.DataArray 'TEMP' (L: 122, z_t: 35, nlat: 140, nlon: 195)>
[116571000 values with dtype=float64]
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    Y        int64 1954
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon

## Calculate Φ using 13 different Eo values and same Ac value

In [6]:
Ac_dist = [10]
Eo_dist = np.arange(-.2,1.1,.1)
print(Ac_dist)
print(Eo_dist)

[10]
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


In [7]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'Phi_{nrow*i+j}'] = mi.Phi(pO2, Temp, Ac, Eo, dEodT=0.022)
    # = xr_reshape(ppphi,'time',['year','mon'],[np.arange(70),np.arange(12)]).mean(dim='mon')

10 -0.2
10 -0.1
10 0.0
10 0.10000000000000003
10 0.2
10 0.3
10 0.4000000000000001
10 0.5
10 0.6000000000000001
10 0.7
10 0.8
10 0.9000000000000001
10 1.0000000000000002
CPU times: user 22.1 s, sys: 58.6 s, total: 1min 20s
Wall time: 1min 50s


In [8]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, L: 122)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    Y        int64 1954
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables: (12/13)
    Phi_0    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_1    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_2    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_3    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_4    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_5    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    ...       ...
    Phi_7    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_8    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_9    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_10   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_11   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_12   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [9]:
%%time
dss.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_ens_mean_monthly.nc', mode='w')

CPU times: user 65.7 ms, sys: 4.42 s, total: 4.49 s
Wall time: 7.87 s
